In [1]:
import tensorflow as tf
import numpy as np
import random
import math
import matplotlib.pyplot as plt
from collections import deque
import numba
import gc
import itertools # Para islice ao calcular média da perda

print(f"TensorFlow Version: {tf.__version__}")
print(f"Numba Version: {numba.__version__}")
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f"GPUs disponíveis: {len(gpus)}. Memória configurada para crescimento dinâmico.")
    except RuntimeError as e:
        print(f"Erro ao configurar GPUs: {e}")
else:
    print("Nenhuma GPU disponível. Rodando em CPU.")

# --- Funções de Lógica do Jogo Otimizadas (sem alterações nesta seção) ---
def novo_jogo(n):
    return np.zeros((n, n), dtype=np.int32)

@numba.jit(nopython=True)
def mover_para_esquerda_numba(mat_np):
    nova = np.zeros_like(mat_np, dtype=mat_np.dtype)
    feito = False
    rows, cols = mat_np.shape
    for i in range(rows):
        contagem = 0
        for j in range(cols):
            if mat_np[i, j] != 0:
                if j != contagem:
                    feito = True
                nova[i, contagem] = mat_np[i, j]
                contagem += 1
    return nova, feito

@numba.jit(nopython=True)
def mesclar_numba(mat_np):
    feito = False
    pontuacao = 0
    rows, cols = mat_np.shape
    for i in range(rows):
        for j in range(cols - 1):
            if mat_np[i, j] != 0 and mat_np[i, j] == mat_np[i, j + 1]:
                val_dobrado = mat_np[i, j] * 2
                mat_np[i, j] = val_dobrado
                pontuacao += val_dobrado
                mat_np[i, j + 1] = 0
                feito = True
    return mat_np, feito, pontuacao

def reverter_opt(mat_np):
    return np.ascontiguousarray(mat_np[:, ::-1])

def transpor_opt(mat_np):
    return np.ascontiguousarray(mat_np.T)

def esquerda_opt(jogo_np):
    jogo_original = np.copy(jogo_np)
    tabuleiro_movido, _ = mover_para_esquerda_numba(jogo_np)
    tabuleiro_mesclado, _, pont_add = mesclar_numba(np.copy(tabuleiro_movido))
    tabuleiro_final, _ = mover_para_esquerda_numba(tabuleiro_mesclado)
    mudou = not np.array_equal(jogo_original, tabuleiro_final)
    return tabuleiro_final, mudou, pont_add

def direita_opt(jogo_np):
    jogo_original = np.copy(jogo_np)
    tabuleiro_invertido = reverter_opt(jogo_np)
    tabuleiro_movido, _ = mover_para_esquerda_numba(tabuleiro_invertido)
    tabuleiro_mesclado, _, pont_add = mesclar_numba(np.copy(tabuleiro_movido))
    tabuleiro_final_mov, _ = mover_para_esquerda_numba(tabuleiro_mesclado)
    tabuleiro_final = reverter_opt(tabuleiro_final_mov)
    mudou = not np.array_equal(jogo_original, tabuleiro_final)
    return tabuleiro_final, mudou, pont_add

def cima_opt(jogo_np):
    jogo_original = np.copy(jogo_np)
    tabuleiro_transposto = transpor_opt(jogo_np)
    tabuleiro_movido, _ = mover_para_esquerda_numba(tabuleiro_transposto)
    tabuleiro_mesclado, _, pont_add = mesclar_numba(np.copy(tabuleiro_movido))
    tabuleiro_final_mov, _ = mover_para_esquerda_numba(tabuleiro_mesclado)
    tabuleiro_final = transpor_opt(tabuleiro_final_mov)
    mudou = not np.array_equal(jogo_original, tabuleiro_final)
    return tabuleiro_final, mudou, pont_add

def baixo_opt(jogo_np):
    jogo_original = np.copy(jogo_np)
    tabuleiro_transposto_invertido = reverter_opt(transpor_opt(jogo_np))
    tabuleiro_movido, _ = mover_para_esquerda_numba(tabuleiro_transposto_invertido)
    tabuleiro_mesclado, _, pont_add = mesclar_numba(np.copy(tabuleiro_movido))
    tabuleiro_final_mov, _ = mover_para_esquerda_numba(tabuleiro_mesclado)
    tabuleiro_final = transpor_opt(reverter_opt(tabuleiro_final_mov))
    mudou = not np.array_equal(jogo_original, tabuleiro_final)
    return tabuleiro_final, mudou, pont_add

controles_opt = {0: cima_opt, 1: esquerda_opt, 2: direita_opt, 3: baixo_opt}

def adiciona_dois_opt(mat_np):
    linhas_vazias, colunas_vazias = np.where(mat_np == 0)
    if len(linhas_vazias) == 0:
        return mat_np
    idx_aleatorio = random.randrange(len(linhas_vazias))
    i, j = linhas_vazias[idx_aleatorio], colunas_vazias[idx_aleatorio]
    mat_np[i, j] = 4 if random.random() >= 0.9 else 2
    return mat_np

@numba.jit(nopython=True)
def estado_jogo_numba(mat_np):
    rows, cols = mat_np.shape
    tem_zero = False
    for r in range(rows):
        for c in range(cols):
            if mat_np[r, c] == 0:
                tem_zero = True
                break
        if tem_zero:
            break
    if tem_zero:
        return 'nao terminou'

    for r in range(rows):
        for c in range(cols - 1):
            if mat_np[r, c] == mat_np[r, c + 1]:
                return 'nao terminou'
    for c in range(cols):
        for r in range(rows - 1):
            if mat_np[r, c] == mat_np[r + 1, c]:
                return 'nao terminou'
    return 'perdeu'

def alterar_valores_opt(X_board_np):
    matriz_potencias = np.zeros(shape=(1, 4, 4, 16), dtype=np.float32)
    rows_idx, cols_idx = np.indices(X_board_np.shape)
    mask_zeros = (X_board_np == 0)
    matriz_potencias[0, rows_idx[mask_zeros], cols_idx[mask_zeros], 0] = 1.0
    mask_non_zeros = (X_board_np != 0)
    if np.any(mask_non_zeros):
        non_zero_values = X_board_np[mask_non_zeros]
        power_indices = np.log2(non_zero_values).astype(np.int32)
        matriz_potencias[0, rows_idx[mask_non_zeros], cols_idx[mask_non_zeros], power_indices] = 1.0
    return matriz_potencias

# --- Hiperparâmetros e Configuração da Rede ---
taxa_aprendizado_inicial = 0.0005
desconto = 0.95
capacidade_memoria = 10000
tamanho_lote = 128
unidades_ocultas = 512 # MODIFICAÇÃO: Aumentado para 512
dropout_rate = 0.5 # MODIFICAÇÃO: Taxa de Dropout
unidades_saida = 4
M = 20000

epsilon_inicial = 1.0
epsilon_final = 0.01
decay_episodios = int(M * 0.5)

memoria_replay = deque(maxlen=capacidade_memoria)
rotulos_replay = deque(maxlen=capacidade_memoria)
perdas_treinamento = deque(maxlen=10000) 
lista_pontuacoes = []
avg_max_q_log = [] # MODIFICAÇÃO: Lista para logar Avg Max Q

inicializador = tf.keras.initializers.HeNormal()

# Camadas convolucionais (sem alteração)
conv1_layer1 = tf.keras.layers.Conv2D(filters=128, kernel_size=(1, 2), padding='valid', activation='relu', use_bias=False, kernel_initializer=inicializador)
conv2_layer1 = tf.keras.layers.Conv2D(filters=128, kernel_size=(2, 1), padding='valid', activation='relu', use_bias=False, kernel_initializer=inicializador)
conv1_layer2 = tf.keras.layers.Conv2D(filters=128, kernel_size=(1, 2), padding='valid', activation='relu', use_bias=False, kernel_initializer=inicializador)
conv2_layer2 = tf.keras.layers.Conv2D(filters=128, kernel_size=(2, 1), padding='valid', activation='relu', use_bias=False, kernel_initializer=inicializador)

def construir_modelo(tensor_entrada):
    saida_conv1 = conv1_layer1(tensor_entrada)
    saida_conv2 = conv2_layer1(tensor_entrada)
    saida_conv11 = conv1_layer2(saida_conv1)
    saida_conv12 = conv2_layer2(saida_conv1)
    saida_conv21 = conv1_layer2(saida_conv2)
    saida_conv22 = conv2_layer2(saida_conv2)
    flatten_camadas = [
        tf.keras.layers.Flatten()(saida_conv1), tf.keras.layers.Flatten()(saida_conv2),
        tf.keras.layers.Flatten()(saida_conv11), tf.keras.layers.Flatten()(saida_conv12),
        tf.keras.layers.Flatten()(saida_conv21), tf.keras.layers.Flatten()(saida_conv22),
    ]
    concatenado = tf.keras.layers.Concatenate(axis=-1)(flatten_camadas)
    # MODIFICAÇÃO: Camada densa aumentada e Dropout adicionado
    densa_oculta = tf.keras.layers.Dense(unidades_ocultas, activation='relu', kernel_initializer=inicializador)(concatenado)
    dropout_layer = tf.keras.layers.Dropout(dropout_rate)(densa_oculta) # Aplicar Dropout aqui
    camada_saida = tf.keras.layers.Dense(unidades_saida, kernel_initializer=inicializador)(dropout_layer) # Dropout antes da saída
    return camada_saida

input_tensor = tf.keras.Input(shape=(4, 4, 16))
output_tensor = construir_modelo(input_tensor)
modelo = tf.keras.Model(inputs=input_tensor, outputs=output_tensor)
modelo.summary()

# MODIFICAÇÃO: Agendamento da Taxa de Aprendizado para CosineDecayRestarts (Warm Restarts)
# Estimativa de passos por ciclo: se um episódio tem ~150-200 passos de treino, 20000 passos = ~100-133 episódios
# Ajuste first_decay_steps conforme achar melhor para a duração do primeiro ciclo.
lr_schedule = tf.keras.optimizers.schedules.CosineDecayRestarts(
    initial_learning_rate=taxa_aprendizado_inicial,
    first_decay_steps=20000,  # Número de passos para o primeiro ciclo de decaimento
    t_mul=2.0,              # Multiplicador para a duração dos ciclos seguintes (2.0 -> cada ciclo é 2x mais longo)
    m_mul=0.9,              # Multiplicador para a taxa de aprendizado no início de cada reinício (0.9 -> LR reinicia 10% menor)
    alpha=0.0001             # Taxa de aprendizado mínima como fração da inicial (0.0001 * 0.0005 = 5e-8)
)
otimizador = tf.keras.optimizers.Adam(learning_rate=lr_schedule, clipnorm=1.0) # Adicionado clipnorm
funcao_perda = tf.keras.losses.MeanSquaredError()

@tf.function
def train_step(model_instance, lote_estados_tensor, lote_rotulos_tensor, loss_fn, optimizer_instance):
    with tf.GradientTape() as tape:
        # training=True é crucial para que o Dropout seja aplicado durante o treino
        predicoes_q = model_instance(lote_estados_tensor, training=True)
        perda = loss_fn(lote_rotulos_tensor, predicoes_q)
    gradientes = tape.gradient(perda, model_instance.trainable_variables)
    optimizer_instance.apply_gradients(zip(gradientes, model_instance.trainable_variables))
    return perda

maior_bloco_geral = 0
melhor_pontuacao = -float('inf')
episodio_melhor_pontuacao = 0

NOME_MODELO_PESOS = "modelo_2048_otimizado_v2_pesos.weights.h5" # Novo nome para pesos
try:
    modelo.load_weights(NOME_MODELO_PESOS)
    print(f"Pesos do modelo '{NOME_MODELO_PESOS}' carregados com sucesso.")
except Exception as e:
    print(f"Não foi possível carregar os pesos do modelo '{NOME_MODELO_PESOS}'. Iniciando do zero. Erro: {e}")

for ep in range(M):
    tabuleiro = novo_jogo(4)
    tabuleiro = adiciona_dois_opt(tabuleiro)
    tabuleiro = adiciona_dois_opt(tabuleiro)
    
    fim_jogo = 'nao terminou'
    pontuacao_total_ep = 0
    iter_local = 0 # MODIFICAÇÃO: Será logado
    
    epsilon = epsilon_final + (epsilon_inicial - epsilon_final) * math.exp(-1. * ep / decay_episodios)

    while fim_jogo == 'nao terminou':
        iter_local += 1
        tabuleiro_antes_acao = np.copy(tabuleiro)
        estado_atual_nn = alterar_valores_opt(tabuleiro_antes_acao)

        # training=False para inferência (desativa Dropout)
        q_values_atuais = modelo(estado_atual_nn, training=False).numpy()[0]
        
        if random.random() < epsilon:
            acao_idx = random.randrange(unidades_saida)
        else:
            acao_idx = np.argmax(q_values_atuais)

        tabuleiro_depois_acao, mudou_estado, recompensa_imediata_mov = controles_opt[acao_idx](np.copy(tabuleiro_antes_acao))
        pontuacao_total_ep += recompensa_imediata_mov
        recompensa_transicao = recompensa_imediata_mov
        if not mudou_estado and estado_jogo_numba(tabuleiro_antes_acao) != 'perdeu':
            recompensa_transicao -= 0.5

        tabuleiro = tabuleiro_depois_acao
        fim_jogo = estado_jogo_numba(tabuleiro)

        if fim_jogo == 'nao terminou' and mudou_estado:
             tabuleiro = adiciona_dois_opt(tabuleiro)
             fim_jogo = estado_jogo_numba(tabuleiro)

        estado_proximo_nn = alterar_valores_opt(tabuleiro)
        
        q_alvo_para_memoria = np.copy(q_values_atuais)
        if fim_jogo == 'perdeu':
            q_alvo_para_memoria[acao_idx] = recompensa_transicao
        else:
            # training=False para inferência (desativa Dropout)
            q_values_proximo_estado = modelo(estado_proximo_nn, training=False).numpy()[0]
            q_alvo_para_memoria[acao_idx] = recompensa_transicao + desconto * np.max(q_values_proximo_estado)
        
        memoria_replay.append(estado_atual_nn)
        rotulos_replay.append(q_alvo_para_memoria)

        if len(memoria_replay) >= tamanho_lote:
            indices_lote = random.sample(range(len(memoria_replay)), tamanho_lote)
            lote_estados_list = [memoria_replay[i] for i in indices_lote]
            lote_rotulos_list = [rotulos_replay[i] for i in indices_lote]
            lote_estados_np = np.vstack(lote_estados_list)
            if lote_estados_np.ndim == 5 and lote_estados_np.shape[1] == 1:
                 lote_estados_np = lote_estados_np.reshape(tamanho_lote, 4, 4, 16)
            lote_rotulos_np = np.array(lote_rotulos_list, dtype=np.float32)
            lote_estados_tensor = tf.convert_to_tensor(lote_estados_np, dtype=tf.float32)
            lote_rotulos_tensor = tf.convert_to_tensor(lote_rotulos_np, dtype=tf.float32)
            
            perda_batch = train_step(modelo, lote_estados_tensor, lote_rotulos_tensor, funcao_perda, otimizador)
            perdas_treinamento.append(perda_batch.numpy())

        if iter_local > 2500: # Limite de passos por episódio
            if fim_jogo == 'nao terminou':
                 fim_jogo = 'perdeu'

        if fim_jogo != 'nao terminou':
            break
    
    # Opcional: Coleta de lixo periódica
    # if (ep + 1) % 100 == 0:
    #     num_collected = gc.collect()
    #     # print(f"--- gc.collect() no final do episódio {ep+1}, objetos coletados: {num_collected} ---")

    lista_pontuacoes.append(pontuacao_total_ep)
    maior_bloco_neste_episodio = np.max(tabuleiro) if np.any(tabuleiro) else 0
    if maior_bloco_neste_episodio > maior_bloco_geral:
        maior_bloco_geral = maior_bloco_neste_episodio

    if pontuacao_total_ep > melhor_pontuacao:
        melhor_pontuacao = pontuacao_total_ep
        episodio_melhor_pontuacao = ep
        print(f"*** Novo Melhor Score: {melhor_pontuacao:.0f} no episódio {ep} ***")
        modelo.save_weights(NOME_MODELO_PESOS)

    # MODIFICAÇÃO: Logging a cada 50 episódios, incluindo novas métricas
    if (ep + 1) % 50 == 0:
        current_lr_val = lr_schedule(otimizador.iterations).numpy()
        
        perda_media_recente_str = 'N/A'
        if perdas_treinamento:
            num_elementos_para_media = min(len(perdas_treinamento), 100)
            if num_elementos_para_media > 0:
                ultimas_n_perdas_para_media = list(itertools.islice(perdas_treinamento, len(perdas_treinamento) - num_elementos_para_media, len(perdas_treinamento)))
                if ultimas_n_perdas_para_media: # Checagem extra
                     perda_media_recente_str = f"{np.mean(ultimas_n_perdas_para_media):.4f}"
        
        avg_max_q_str = 'N/A'
        if len(memoria_replay) >= tamanho_lote: # Só calcula se tiver amostras suficientes
            sample_indices_q = random.sample(range(len(memoria_replay)), min(len(memoria_replay), 32))
            sample_states_q = [memoria_replay[i] for i in sample_indices_q]
            if sample_states_q:
                # training=False para inferência (desativa Dropout)
                max_q_values_sample = [np.max(modelo(s, training=False).numpy()) for s in sample_states_q]
                avg_max_q_val = np.mean(max_q_values_sample)
                avg_max_q_str = f"{avg_max_q_val:.2f}"
                avg_max_q_log.append(avg_max_q_val) # Armazena para plot futuro se desejado

        print(f"Ep: {ep+1}/{M} | Score: {pontuacao_total_ep:.0f} | Passos: {iter_local} | Maior Bloco: {maior_bloco_neste_episodio} "
              f"| Epsilon: {epsilon:.3f} | LR: {current_lr_val:.7f} " # Aumentado precisão da LR
              f"| Mem: {len(memoria_replay)} | Perda(100): {perda_media_recente_str} | AvgMaxQ: {avg_max_q_str}")

    if (ep + 1) % 500 == 0:
        modelo.save_weights(f"modelo_2048_backup_ep{ep+1}.weights.h5")

print(f"\nTreinamento Concluído!")
print(f"Melhor Score Obtido: {melhor_pontuacao:.0f} no Episódio: {episodio_melhor_pontuacao}")
modelo.save(f"modelo_2048_final_completo.keras")
print(f"Modelo completo final salvo como modelo_2048_final_completo.keras")

plt.figure(figsize=(18, 12)) # Aumentado tamanho da figura
plt.subplot(2, 2, 1) # Ajustado para 2x2 grid
plt.plot(lista_pontuacoes, label='Pontuação por Episódio')
if len(lista_pontuacoes) >= 100:
    pontuacoes_ma = np.convolve(lista_pontuacoes, np.ones(100)/100, mode='valid')
    plt.plot(np.arange(len(pontuacoes_ma)) + 99, pontuacoes_ma, label='Média Móvel (100 ep.)', color='red', alpha=0.7)
plt.xlabel("Episódio")
plt.ylabel("Pontuação")
plt.title("Evolução da Pontuação")
plt.legend()
plt.grid(True)

if perdas_treinamento:
    perdas_lista_para_plot = list(perdas_treinamento)
    plt.subplot(2, 2, 2) # Ajustado para 2x2 grid
    plt.plot(perdas_lista_para_plot, label='Perda por Batch', alpha=0.3)
    if len(perdas_lista_para_plot) >= 500:
        perdas_ma = np.convolve(perdas_lista_para_plot, np.ones(500)/500, mode='valid')
        plt.plot(np.arange(len(perdas_ma)) + 499, perdas_ma, label='Média Móvel (500 batches)', color='orange')
    plt.xlabel("Número do Batch de Treinamento")
    plt.ylabel("Perda (MSE)")
    plt.title("Evolução da Perda")
    plt.yscale('log')
    plt.legend()
    plt.grid(True)

# MODIFICAÇÃO: Plot do Avg Max Q-Value
if avg_max_q_log:
    plt.subplot(2, 2, 3) # Ajustado para 2x2 grid
    # Os valores de AvgMaxQ são logados a cada 50 episódios. O eixo x precisa refletir isso.
    episodios_avg_max_q = np.arange(len(avg_max_q_log)) * 50 
    plt.plot(episodios_avg_max_q, avg_max_q_log, label='Avg Max Q-Value (amostra)', color='green')
    plt.xlabel("Episódio (aproximado, log a cada 50 ep)")
    plt.ylabel("Avg Max Q-Value")
    plt.title("Evolução do Avg Max Q-Value")
    plt.legend()
    plt.grid(True)

# Placeholder para um quarto gráfico, se necessário, ou pode remover
plt.subplot(2, 2, 4) 
plt.text(0.5, 0.5, 'Outro Gráfico (ex: Duração do Episódio)', horizontalalignment='center', verticalalignment='center')
plt.title("Métrica Adicional")


plt.tight_layout()
plt.show()

2025-07-11 00:39:11.799782: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-11 00:39:11.867632: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-11 00:39:12.119100: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-07-11 00:39:12.989096: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


TensorFlow Version: 2.16.1
Numba Version: 0.61.2
Nenhuma GPU disponível. Rodando em CPU.


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 4, 4, 16)  │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 4, 3, 128) │      4,096 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 3, 4, 128) │      4,096 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 3, 3, 128) │     32,768 │ conv2d[0][0],     │
│                     │                   │            │ conv2d_1[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 2, 4, 128) │     32,768 │ conv2d[0][0],     │
│                     │                   │            │ conv2d_1[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten (Flatten)   │ (None, 1536)      │          0 │ conv2d[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_1 (Flatten) │ (None, 1536)      │          0 │ conv2d_1[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_2 (Flatten) │ (None, 1024)      │          0 │ conv2d_2[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_3 (Flatten) │ (None, 1152)      │          0 │ conv2d_3[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_4 (Flatten) │ (None, 1152)      │          0 │ conv2d_2[1][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_5 (Flatten) │ (None, 1024)      │          0 │ conv2d_3[1][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 7424)      │          0 │ flatten[0][0],    │
│ (Concatenate)       │                   │            │ flatten_1[0][0],  │
│                     │                   │            │ flatten_2[0][0],  │
│                     │                   │            │ flatten_3[0][0],  │
│                     │                   │            │ flatten_4[0][0],  │
│                     │                   │            │ flatten_5[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 512)       │  3,801,600 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 512)       │          0 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 4)         │      2,052 │ dropout[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 3,877,380 (14.79 MB)

 Trainable params: 3,877,380 (14.79 MB)

 Non-trainable params: 0 (0.00 B)

Pesos do modelo 'modelo_2048_otimizado_v2_pesos.weights.h5' carregados com sucesso.
*** Novo Melhor Score: 1388 no episódio 0 ***


KeyboardInterrupt: 